In [1]:
import csv
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
import math
import time
from time import sleep
from random import randint

In [2]:
start_time = time.time()

In [3]:
baselink = "https://ca.indeed.com"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}

In [4]:
df = pd.DataFrame(columns=['Title', 'Company', 'Location', 'JobLink',
                           'CompanyLink', 'DatePosted', 'ExtractDate',
                           'Description', 'Salary', 'JobType', 'Remote', 'Reviews', 'Stars'])

In [5]:
# Generate url from position, location and date posted.
def get_url(position, location, dateposted):
    template = 'https://ca.indeed.com/jobs?q={}&l={}&fromage={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    dateposted = dateposted.replace(' ', '+')
    url = template.format(position, location, dateposted)
    return url
url = get_url('data analyst', 'canada', '7')
print(url)

https://ca.indeed.com/jobs?q=data+analyst&l=canada&fromage=7


In [8]:
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.content, 'html.parser')
pages = soup.find("div", attrs={"id": "searchCountPages"})
#pages = pages.replace("Page 1 of ", "")
#pages = pages.replace(" jobs", "")
#pages = pages.replace(",", "")
#X = (int(math.ceil(int(pages)/14)))*10
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<title>Access denied</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/errors.css" media="screen" rel="stylesheet"/>
<script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLHttpRequest;a={event:"feedback clicked",properties:{errorCode:1020,helpful:a,version:3}};b.open("POST","https://sparrow.cloudflare.com/api/v1/event");b.setRequestHeader("Content-Type","application/json");b.setRequestHeader("Sparrow-Source-Key","c771f0e4b54944bebf4261d44bd79a1e");
b.send(JSON.stringify(a));c.classList.add("feedback-hidden");d.classList.remove("feedback-hidden")};document.addEventListener("DOMContentLoaded",function(){var a=docume

In [7]:
for number in range(0, 100, 10):
    links = url+'&start={}'.format(number)
    r = requests.get(links, headers = headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    jobcards = soup.find("div", attrs={"id": "mosaic-zone-jobcards"})
    links = []
    
    for l in jobcards.find_all('a', attrs={'href': re.compile("js")}):
        links.append(l.get('href'))
        
    for n in links:
        joblink = baselink + str(n)
        r = requests.get(joblink, headers = headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        extract_date = datetime.today().strftime('%Y-%m-%d')
        loc = soup.find(class_="icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle")
        com = soup.find(class_="jobsearch-CompanyInfoContainer")
        sal = soup.find(class_="jobsearch-JobMetadataHeader-item")
        rat = soup.find(class_="icl-u-lg-block icl-u-lg-mr--sm icl-u-xs-hide")
        pos = soup.find(class_="jobsearch-HiringInsights-entry--text")
        sleep(randint(2,4))
        
        try:
            reviews = rat.find(
                'div', string=re.compile("reviews")).text.strip()
        except AttributeError:
            reviews = "NA"
                        
        try:
            stars = rat.select_one('meta[content]')['content']
        except AttributeError:
            stars = "NA"
                        
        try:
            desc = soup.find(
                class_="jobsearch-jobDescriptionText").text.strip()
        except AttributeError:
            desc = "NA"
                        
        try:
            job_title = soup.find(
                class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").text.strip()
        except AttributeError:
            job_title = "NA"
                        
        try:
            job_type = sal.find('span', string=re.compile(
                "Full|Part|Casu|Intern|Perman|Contr|term|Temp|Freel|Apprent")).text.strip()
        except AttributeError:
            job_type = "NA"
            
        try:
            remote = loc.find(
                'div', string=re.compile("Remote|remote")).text.strip()
        except AttributeError:
            remote = "NA"
                        
        try:
            salary = sal.find(
                'span', string=re.compile("year|hour")).text.strip()
        except AttributeError:
            salary = "NA"
                        
        try:
            companylink = com.find(
                'a', attrs={'href': re.compile("//")}).get('href')
        except AttributeError:
            companylink = "NA"
                        
        try:
            company = com.find(
                'a', attrs={'href': re.compile("//")}).string.strip()
        except AttributeError:
            company = "NA"
                        
        try:
            location = loc.find('div', string=re.compile(
                "ON|SK|BC|AB|PE|NS|MB|QC|NL|PE|YT|NB|NT|NU")).string.strip()
        except AttributeError:
            location = "NA"
            
        try:
            post = re.sub('\D', '', pos.get_text().strip())
        except AttributeError:
            post = ''
    
        if post.isdigit():
            posted = int(post)
        else:
            posted = int(0) 
            
        try:
            dateposted = (datetime.today() - timedelta(days=posted)).strftime('%Y-%m-%d')
        except AttributeError:
            dateposted = "NA"
                        
        df.loc[len(df.index)] = [job_title, company, location, joblink, companylink, dateposted, extract_date, desc, salary, job_type, remote, reviews, stars]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
date = datetime.today().strftime("%Y-%m-%d-%H-%M")
fileName = "indeedjobs" + date + ".csv" 
print(fileName)

In [ ]:
df.to_csv(fileName, index = False)

In [ ]:
print("--- %s minutes ---" % (math.ceil(int(time.time() - start_time)/60)))

In [ ]:
# Importing dataset:
md = pd.read_csv("indeedjobs2022.csv")
nd = pd.read_csv(fileName)

In [ ]:
mdf = pd.concat([md, df])

In [ ]:
mdf.to_csv("indeedjobs2022.csv", index = False)